In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("spotify_millsongdata.csv")

In [17]:
df.head(5) #first 5 rows

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [18]:
df.tail(5) #last 5 rows

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [19]:
df.isnull().sum()  #checking for null values

artist    0
song      0
link      0
text      0
dtype: int64

In [20]:
df = df.sample(10000).drop('link',axis=1).reset_index(drop=True)

Text Cleaning/ Text Preprocessing

In [21]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n',' ',regex=True)

In [ ]:
df.tail(5)

,artist,song,text
9995,Smiths,Shoplifters Of The World Unite,learn to love me \r assemble the ways \r now...
9996,Utada Hikaru,Eternally,"right before me, you glimmer slightly i can't ..."
9997,J Cole,Blowin Smoke,"[verse:] \r blowing smoke, showing growth, no..."
9998,Kris Kristofferson,Bad Love Story,once again you tried to show me \r what my ey...
9999,Death,Symbolic,i don't mean to dwell \r but i can't help mys...


Tokenizing or Stemming


In [23]:
from nltk.tokenize import punkt
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
  tok = nltk.word_tokenize(txt)
  a= [stemmer.stem(w) for w in tok]
  return " ".join(a)

In [ ]:
df['text'].apply(lambda x: token(x))

In [11]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [14]:
matrix = tfid.fit_transform(df['text'])

In [15]:
sim = cosine_similarity(matrix)

In [ ]:
sim[0]

array([1.        , 0.00772748, 0.03705948, ..., 0.01778575, 0.0011349 ,
       0.0079154 ])

In [ ]:
df[df['song']=='Eternally'].index[0]

9996

Recommender Function


In [16]:
def recommender(song_name):
  idx = df[df['song']==song_name].index[0]
  dist = sorted(list(enumerate(sim[idx])),reverse=True, key = lambda x: x[1])
  song = []
  for s_id in dist[1:10]:
    song.append(df.iloc[s_id[0]].song)
  return song

In [ ]:
recommender('In Your Eyes')

['Lately',
 'Keep Coming Back',
 'My Head Is Spinning',
 'Whole World Is Watching']

In [17]:
import pickle

In [18]:
pickle.dump(sim,open("similarity","wb"))

In [20]:
pickle.dump(df,open("df","wb"))